# Traffic Light Training
---

#### Resources: 
1. [Training on Google Cloud](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/running_pets.md)
2. [Training the model (medium-WuStangDan)](https://medium.com/@WuStangDan/step-by-step-tensorflow-object-detection-api-tutorial-part-4-training-the-model-68a9e5d5a333)
3. [Sample .config file](https://github.com/tensorflow/models/blob/master/research/object_detection/faster_rcnn_inception_resnet_v2_atrous_oid.config)
4. [Reading TFRecord file](http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/)
5. [COCO Trained Models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)
6. [Udacity Transfer Learning Lab](https://github.com/udacity/CarND-Transfer-Learning-Lab/blob/master/feature_extraction_solution.py)
7. [Ronny Restrepo's transfer learning example](http://ronny.rest/blog/post_2017_10_13_tf_transfer_learning/)

#### Training instruction:
- Get Tensorflow Models (place in same directory)
- Download Bosch Traffic Light Dataset (or any dataset)
- Create train.record and test.record file from data_processing.ipynb
- Download a trained model (faster_rcnn_resnet101) from [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)
- Copy the train_data folder in this folder (place in same directory)
- The folder should have the following files
    * train.record
    * test.record
    * model.ckpt-0.data-00000-of-00001
    * model.ckpt-0.index
    * model.ckpt-0.meta
    * train.py
    * label_map.pbtx
    * faster_rcnn_resnet101_coco.config
- On terminal, go to the train_data directory and run following line  

```shell
>> python train.py --logtostderr --train_dir=./ --pipeline_config_path=faster_rcnn_resnet101_coco.config
```


## Environment Preparation

In [1]:
# imports
import numpy as np
import os
import sys
from PIL import Image
import tensorflow as tf
import skimage.io as io
from matplotlib import pyplot as plt

# This is needed to display the images.
%matplotlib inline

# Dataset paths
PATH_TO_GREEN_IMGS = "../../dataset/images_for_training/green/"
PATH_TO_RED_IMGS = "../../dataset/images_for_training/red/"
PATH_TO_YELLOW_IMGS = "../../dataset/images_for_training/yellow/"
PATH_TO_XML = "../../dataset/images_for_training/"
OUTPUT_PATH = os.path.dirname("../../output_data/")

# Place TF Models repository on the same directory (https://github.com/tensorflow/models)
sys.path.append("../../models/research/")
sys.path.append("../../models/research/slim/") # deployment
sys.path.append("../../models/research/object_detection/")

# Check TF File

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
data_path = '../../train_data/bosch_test.record'

with tf.Session() as sess:
    feature = {'image/encoded': tf.FixedLenFeature([], tf.string)}
    
    print(feature) 
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path], num_epochs=1)
    
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['image/encoded'], tf.int8)
    
    # Reshape image data into the original shape
    #image = tf.reshape(image, [224, 224, 3])
    #img = tf.image.decode_png(image, channels=3)
    # Any preprocessing here ...
    print(image)
    #result = sess.run(image)
    #plt.imshow(result)
    #plt.show()

    # Creates batches by randomly shuffling tensors
    #images, labels = tf.train.shuffle_batch([image, label], batch_size=10, capacity=30, num_threads=1, min_after_dequeue=10)

{'image/encoded': FixedLenFeature(shape=[], dtype=tf.string, default_value=None)}
Tensor("DecodeRaw:0", shape=(?,), dtype=int8)


In [28]:
### Retrieve and Check TF records file ###
tfrecords_filename = 'bosch_test.record'

filename_queue = tf.train.string_input_producer(
    [tfrecords_filename], num_epochs=10)

# Even when reading in multiple threads, share the filename
# queue.
image, annotation = read_and_decode(filename_queue)

# The op for initializing the variables.
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

with tf.Session()  as sess:
    
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # Let's read off 3 batches just for example
    for i in range(1):
    
        img, anno = sess.run([image, annotation])
        print(img[0, :, :, :].shape)
        
        print('current batch')
        
        # We selected the batch size of two
        # So we should get two image pairs in each batch
        # Let's make sure it is random

        io.imshow(img[0, :, :, :])
        io.show()

        io.imshow(anno[0, :, :, 0])
        io.show()
        
        io.imshow(img[1, :, :, :])
        io.show()

        io.imshow(anno[1, :, :, 0])
        io.show()
        
    
    coord.request_stop()
    coord.join(threads)

NameError: name 'read_and_decode' is not defined

## Import COCO dataset

In [ ]:
# pick and download a model

model_name = 'faster_rcnn_resnet101_coco_2017_11_08'


# Assume fc_2nd_last is the 2nd_last fully connected layer in your network and nb_classes is the number of classes in your new dataset.
#shape = (fc_2nd_last.get_shape().as_list()[-1], nb_classes)
#fc_last_W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
#fc_last_b = tf.Variable(tf.zeros(nb_classes))
#logits = tf.nn.xw_plus_b(fc_2nd_last, fc_last_W, fc_last_b)


In [ ]:
# transfer learning Imports
import os
import sys
import glob
import argparse
import matplotlib.pyplot as plt

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
NB_EPOCHS = 3
BAT_SIZE = 32
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172

In [ ]:
### Helper Functions ###
def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
    Args:
    base_model: keras model excluding top
    nb_classes: # of classes
    Returns:
    new keras model with last layer
    """
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    model = Model(input=base_model.input, output=predictions)
    
    return model

def setup_to_finetune(model):
    """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.
    note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch
    Args:
    model: keras model
    """
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


## Train


In [ ]:
### Use transfer learning and fine-tuning to train a network on a new dataset ###

nb_train_samples = get_nb_files(args.train_dir)
nb_val_samples = get_nb_files(args.val_dir)

nb_classes = 4
nb_epoch = 2
batch_size = 2

# data prep
train_datagen =  ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    args.train_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
)

validation_generator = test_datagen.flow_from_directory(
    args.val_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
)

# setup model
base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
model = add_new_last_layer(base_model, nb_classes)

# transfer learning
setup_to_transfer_learn(model, base_model)

history_tl = model.fit_generator(
                            train_generator,
                            nb_epoch=nb_epoch,
                            samples_per_epoch=nb_train_samples,
                            validation_data=validation_generator,
                            nb_val_samples=nb_val_samples,
                            class_weight='auto')

# fine-tuning
setup_to_finetune(model)

history_ft = model.fit_generator(
                            train_generator,
                            samples_per_epoch=nb_train_samples,
                            nb_epoch=nb_epoch,
                            validation_data=validation_generator,
                            nb_val_samples=nb_val_samples,
                            class_weight='auto')

model.save(args.output_model_file)

## Plot

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r.')
plt.plot(epochs, val_acc, 'r')
plt.title('Training and validation accuracy')

plt.figure()
plt.plot(epochs, loss, 'r.')
plt.plot(epochs, val_loss, 'r-')
plt.title('Training and validation loss')
plt.show()